<h1 align='center'><font size =10 color='gold'>Segmentation and Clustering Neighborhood in Toronto</font></h1>
<h3>By Gaurav Bhardwaj</h3>

Importing Modules required for this `Notebook`: 

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import lxml.html as lh
from geopy.geocoders import Nominatim
print('Libraries imported.')

Libraries imported.


Collecting data by using Web Scarping Method.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
print('Data downloaded!')

Data downloaded!


In [3]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [5]:
Dict={title:column for (title,column) in col}
data=pd.DataFrame(Dict)

Visualizing the collected Data

In [6]:
data.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [7]:
data.columns

Index(['Postal Code\n', 'Borough\n', 'Neighborhood\n'], dtype='object')

In [8]:
data['Postal Code\n']=data['Postal Code\n'].astype('string')
data['Borough\n']=data['Borough\n'].astype('string')
data['Neighborhood\n']=data['Neighborhood\n'].astype('string')

In [9]:
data.dtypes

Postal Code\n     string
Borough\n         string
Neighborhood\n    string
dtype: object

Removing Rows containing `Not assigned`,`\n`,` `:

In [10]:
data.replace('\n',' ',regex=True)
data.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
data.drop(data.index[data['Borough\n'] == 'Not assigned\n'], inplace = True)
# Reset the index and dropping the previous index
data = data.reset_index(drop=True)

data.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
data.drop(data.index[data['Postal Code\n'] == ' '], inplace = True)
# Reset the index and dropping the previous index
data = data.reset_index(drop=True)

data.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Renaming the column Headings:

In [13]:
data.columns= ['Postal Code', 'Borough', 'Neighborhood']
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping `Neighborhoods` belonging to same **Postal Code** and **Borough**:

In [14]:
data = data.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
data.head()

,Postal Code,Borough,Neighborhood
0,\n,Canadian postal codes\n,\n
1,M1B\n,Scarborough\n,"Malvern, Rouge\n"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek\n"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill\n"
4,M1G\n,Scarborough\n,Woburn\n


In [15]:
data.loc[data['Neighborhood'] == '\n', 'Neighborhood'] = data['Borough']
data.head()

,Postal Code,Borough,Neighborhood
0,\n,Canadian postal codes\n,Canadian postal codes\n
1,M1B\n,Scarborough\n,"Malvern, Rouge\n"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek\n"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill\n"
4,M1G\n,Scarborough\n,Woburn\n


In [16]:
data = data.replace('\n',' ',regex= True)
data.head()

,Postal Code,Borough,Neighborhood
0,,Canadian postal codes,Canadian postal codes
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [17]:
data.drop(data.index[data['Postal Code'] == ' '], inplace = True)
# Reset the index and dropping the previous index
data = data.reset_index(drop=True)

data.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Closing the data by saving to `.csv` file and Checking it's Shape :

In [18]:
data.to_csv(r"Segmented_DF_Toronto.csv")
print("Shape of Data Frame: ",data.shape)

Shape of Data Frame:  (103, 3)
